<a href="https://colab.research.google.com/github/EncodingDavid/tensorflow/blob/master/%E8%AF%86%E5%88%AB%E6%89%8B%E5%86%99%E5%AD%97%E4%BD%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow实现Softmax Regression识别手写字体
首先对MNIST数据进行加载，Tensorflow为我们提供了一个方便的封装，可以直接加载MNIST数据为我们期望格式

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MINST_data/",one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MINST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MINST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MINST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

# 查看MNIST数据集的情况
可以看到有55000个样本，测试集有10000个样本，验证集有5000个样本。每一个样本都有它对应的标注信息，即label。我们将在训练集上训练模型，在验证集上检验效果并决定何时完成训练，最后我们在测试集评测模型的效果。

In [7]:
print(mnist.train.images.shape, mnist.train.labels.shape)

(55000, 784) (55000, 10)


In [8]:
print(mnist.test.images.shape, mnist.test.labels.shape)

(10000, 784) (10000, 10)


In [9]:
print(mnist.validation.images.shape, mnist.validation.labels.shape)

(5000, 784) (5000, 10)


前面我们提到的图像是28✖️28pixel大小的灰度图片，需要将它变成784维的特征，把一张图片变成一维向量

首先引入TensorFlow库，创建一个新的InteractiveSession，使用这个命令会将这个session注册为默认的session，之后的运算也默认run在这个session里，不同session之间的数据和运算应该都是互相独立的。接下来创建一个Placeholder，即输入数据的地方。Placeholder的第一个参数是数据类型，第二个参数是[None,784]代表tensor的shape，也是数据的尺寸，这里None代表不限条数的输入，784代表每条输入是一个784维的向量。

In [11]:
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32,[None,784])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# 创建Variable对象
10代表10类，我们的数字是0-9之间的，所以有10类
- 使用`Softmax Regression`模型中的weights和biases创建Variable对象
- Variable在模型训练迭代中是持久化的
- 将weights和biases中初始化为0（对于复杂的卷积网络很重要）

In [0]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

** 利用公式$y = softmax(Wx+b)$ **

In [0]:
y = tf.nn.softmax(tf.matmul(x,W)+b)

为了训练模型，我们需要定义一个`loss function`来描述模型对问题的分类精度。loss越小，模型的分类结果与真实值的偏差越小，也就是模型越精确

In [0]:
# 定义cross-entropy
# 先定义一个placeholder，输入是真实的label，用来计算cross-entropy。
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))

只需再定义一个常见的随机梯度下降SGD。定义完了即可自动求导，并根据反向传播算法进行训练，在每一轮迭代时更新参数来减少loss。在后台，TensorFlow会自动添加许多运算操作来实现刚才提到的反向传播和梯度下降，而给我们提供的是一个封装好的优化器。

In [21]:
# 直接调用tf.train.GradientDescentOptimizer,设置学习速率为0.5，优化目标设定为cross-entropy得到进行训练的操作train_step
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


In [0]:
# 使用全局参数优化器tf.global_variables_initializer
tf.global_variables_initializer().run()

In [0]:
# 最后一步，执行训练操作train_step。使用小样本进行训练称为随机梯度下降，与每次使用全部样本的传统的梯度下降对应。
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x: batch_xs, y_: batch_ys})

In [0]:
# 对模型的准确度进行验证
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))

In [0]:
# 统计全部样本预测的accuracy，这里需要先用tf.cast将之前correct_prediction输出的bool值转换为float32，再求平均
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [34]:
# 对MNIST数据进行分类识别，在测试集上平均准确率可达92%左右
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

0.9193


# 总结
通过上面的简单例子，我们使用TensorFlow实现来一个简单的机器学习算法Softmax Regression，这可以算作一个没有隐含层的最浅的神经网络。简单来说，步骤有4部分。
- 定义算法公式，也就是神经网络forward时的计算。
- 定义loss，选定优化器，并指定优化器优化loss
- 迭代地对数据进行训练
- 在测试集或验证集上对准确率进行评测

这四个是TensorFlow进行算法设计、训练的核心步骤。

虽然准确率达到92%，但远远不能达到实用的程度。手写数字的主要应用场景是识别银行支票，如果准确率不够，可能会有大麻烦。后面需要用到多层感知机和卷积神经网络来进一步地提高准确率。